# Calculate biggest donors for County Executive candidates currently in office

In [1]:
from functools import reduce
import glob
import pandas as pd
from PyPDF2 import PdfReader
import re

In [2]:
CONTRIB_REGEX = r'Full Name of Contrib.*\n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(\d+)\n(\d+)\n(\d+)\n'
RECEIPT_REGEX = r'Full Name \n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(\d+)\n(\d+)\n(\d+)\nReceipt Description\n.*\n'
UNITEMIZED_REGEX = r'Unitemized  Contributions Received - \$ 50.00 or Less Per Contributor\nTOTAL for the Reporting Period           \(1\)\n\$\n(.*)\n'

def flatten(l):
    return [item for sublist in l for item in sublist]

def get_contributions(name):
    contributions = []
    for file in glob.glob('input/%s_*.pdf' % name.capitalize()):
        contributions.append(flatten(process_report(file)))

    df = pd.DataFrame(
        flatten(contributions),
        columns=['name', 'address1', 'city', 'state', 'zipcode', 'amount', 'month', 'day', 'year'])
    df['name'] = df.name.str.upper()
    df['amount'] = pd.to_numeric(df.amount.str.replace(',', ''))

    return df

def process_report(filename):
    reader = PdfReader(filename)
    text = ""

    for page in reader.pages:
        text += page.extract_text() + "\n"

    unitemized = re.findall(UNITEMIZED_REGEX, text)
    if len(unitemized) == 0:
        unitemized = []
    else:
        unitemized = [(
            'UNITEMIZED',
            '',
            'PITTSBURGH',
            'PA',
            '',
            unitemized[0],
            '',
            '',
            ''
        )]

    return [
        re.findall(CONTRIB_REGEX, text),
        re.findall(RECEIPT_REGEX, text),
        unitemized
    ]

### John Weinstein (state committee)

In [3]:
df_weinstein = get_contributions('Weinstein')
df_weinstein = df_weinstein.rename(columns={'amount': 'amount_wein'})

df_weinstein['name'] = df_weinstein.name.replace({
    "AMALGAMATED TRANSIT UNION COPE": "ATU COPE VOLUNTARY ACCOUNT",
    "IBEW LOCAL UNION NO. 5 PAC": "LOCAL 0005 IBEW PAC",
    "MIDATLANTIC POLITICAL LEAGUE - MALPA": "MID-ATLANTIC LABORERS' POLITICAL LEAGUE",
    "STEAMFITTERS LOCAL UNON #449": "LOCAL 0449 STEAMFITTERS UNION PAC",
    "STEAMFITTERS LOCAL UNION 449 PAC FUND": "LOCAL 0449 STEAMFITTERS UNION PAC",
    "PLUMBERS LOCAL UNION NO. 27 PAC": "LOCAL 0027 PLUMBERS UNION PAC",
    "BRICKLAYERS &AMP; ALLIED CRAFTWORKERS LOCAL 9 PAC": "LOCAL 0009 BRICKLAYERS & ALLIED CRAFTWORKERS PA PAC",
    "PITTSBURGH FIRE FIGHTERS LOCAL NO 1 FIRE PAC ACCOUNT": "PGH FIRE FIGHTERS LOCAL #1 FIRE PAC",
    "TEAMSTERS LOCAL UNION 249 - DRIVE FUND": "LOCAL 0249 TEAMSTERS DRIVE",
    "U.W.U.A. LOCAL 433 PAC": "LOCAL 0433 UWUA (UTILITY WORKERS)",
    "TEAMSTER JOINT COUNCIL 40 PAC": "TEAMSTERS JT COUNCIL 40 PAC",
    "AFSCME COUNCIL 13 POLITICAL &AMP; LEGISLATIVE": "AFSCME COUNCIL 13 POL & LEG ACCT",
    # Laborers
    "LABORERS DISTRICT COUNCIL OF WESTERN PENNSYLVANIA": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
    "LABORERS' DISTRICT COUNCIL OF WESTERN PENNSYLVANIA": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
    "WESTERN PA LABORERS UNION PAC": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
    "WESTERN PENNSYLVANIA LABORERS": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
    "WESTERN PENNSYLVANIA LABORERS' PAC": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
})

In [4]:
df_weinstein_topcontribs = df_weinstein.groupby('name').amount_wein.sum().to_frame().reset_index()

weinstein_sum = df_weinstein.amount_wein.sum()
df_weinstein_topcontribs['pct_wein'] = df_weinstein_topcontribs.amount_wein / weinstein_sum

df_weinstein_topcontribs.sort_values(by='amount_wein', ascending=False).head(10)

,name,amount_wein,pct_wein
321,WESTERN PENNSYLVANIA LABORERS 2019 PAC,90000.0,0.180471
22,ATU COPE VOLUNTARY ACCOUNT,37614.0,0.075425
103,FNB CORPORATION PAC,22500.0,0.045118
241,MID-ATLANTIC LABORERS' POLITICAL LEAGUE,22000.0,0.044115
113,FRIENDS OF RANDY MARTINI,20000.0,0.040105
203,LOCAL 0449 STEAMFITTERS UNION PAC,15000.0,0.030078
198,LOCAL 0005 IBEW PAC,10000.0,0.020052
10,AMALGAMATED TRANSIT UNION - LOCAL 85,10000.0,0.020052
146,JAMES P. GRANT,10000.0,0.020052
41,CHARLES HAMMEL III,7500.0,0.015039


### Sara Innamorato (state committee)

In [5]:
df_innamorato = get_contributions('Innamorato')
df_innamorato = df_innamorato.rename(columns={'amount': 'amount_inna'})

df_innamorato['name'] = df_innamorato.name.replace({
    "1776 PAC": "1776 PAC (UFCW)",
    "AFSCME COUNCIL 13": "AFSCME COUNCIL 13 POL & LEG ACCT",
    "EVAN SEGAL": "EVAN J. SEGAL",
    "PENNSYLVANIA SEIU COPE": "SEIU HEALTHCARE PA COPE",
    "PFT POLITICAL ACTION FUND": "PFT POL ACTION FUND (PGH FED TEACHERS)",
    "TEAMSTERS LOCAL UNION 249": "LOCAL 0249 TEAMSTERS DRIVE",
    "TEAMSERS LOCAL UNION 249": "LOCAL 0249 TEAMSTERS DRIVE",
    "TEAMSTERS LOCAL UNION 249 DRIVE FUND": "LOCAL 0249 TEAMSTERS DRIVE",
    "WESTERN PENNSYLVANIA LABORERS POLITICAL ACTION FUND": "WESTERN PENNSYLVANIA LABORERS 2019 PAC",
})

In [6]:
df_innamorato_topcontribs = df_innamorato.groupby('name').amount_inna.sum().to_frame().reset_index()

innamorato_sum = df_innamorato.amount_inna.sum()
df_innamorato_topcontribs['pct_inna'] = df_innamorato_topcontribs.amount_inna / innamorato_sum

df_innamorato_topcontribs.sort_values(by='amount_inna', ascending=False).head(10)

,name,amount_inna,pct_inna
629,UNITEMIZED,35293.34,0.109579
206,EVAN J. SEGAL,16020.63,0.049741
590,SEIU HEALTHCARE PA COPE,11250.00,0.034929
519,PENNSYLVANIA SIERRA CLUB PAC,10000.00,0.031048
154,DAVID TURNER,7650.00,0.023752
518,PENNSYLVANIA DEMOCRATIC PARTY,7613.90,0.023640
650,WOMEN FOR THE FUTURE,6500.00,0.020181
474,NANCY BERNSTEIN,6500.00,0.020181
18,ALLEGHENY COUNTY DEMOCRATIC DELEGATION,5750.00,0.017853
550,REPRESENT PAC,5500.00,0.017076


### Michael Lamb (state and local committees)

In [7]:
namespre22 = [
    'filer_id', 'election_year', 'cycle',
    'name', 'address1', 'address2', 'city', 'state', 'zipcode',
    'date', 'amount_lamb', 'description'
]

names2022 = [
    'report_id', 'filer_id', 'election_year', 'filer_date', 'cycle',
    'name', 'address1', 'address2', 'city', 'state', 'zipcode',
    'date', 'amount_lamb', 'description'
]

expenses = pd.concat([
    pd.read_csv('input/state_expense_2017.csv', encoding='unicode_escape', names=namespre22),
    pd.read_csv('input/state_expense_2018.csv', encoding='unicode_escape', names=namespre22, skiprows=[1333, 22823, 62725, 108098]),
    pd.read_csv('input/state_expense_2019.csv', encoding='unicode_escape', names=namespre22),
    pd.read_csv('input/state_expense_2020.csv', encoding='unicode_escape', names=namespre22, skiprows=[60172]),
    pd.read_csv('input/state_expense_2021.csv', encoding='unicode_escape', names=namespre22, skiprows=[14403, 66485, 66486, 66487, 66488, 66489, 66490, 66491, 87279]),
    pd.read_csv('input/state_expense_2022.csv', encoding='unicode_escape', usecols=names2022, names=names2022)
])

expenses.loc[expenses.description.isna(), 'description'] = ''
expenses.loc[expenses.name.isna(), 'name'] = ''

expenses['description'] = expenses.description.str.upper()
expenses['name'] = expenses.name.str.upper()
expenses['address1'] = expenses.address1.str.upper()

### Remove malformated vendor
expenses = expenses.drop(expenses[expenses['name'] == 'NCH SOFTWARE'].index)

In [8]:
df_lamb = expenses[
    (
        ~expenses.name.isin([
            'FRIENDS OF MICHAEL LAMBERT',
            'FRIENDS OF RICH FITZGERALD'
            'LAW OFFICE OF MICHAEL LAMBERT LLC',
            'MARINE CORPS LEAGUE',
        ])
    ) & (
        (expenses.name.str.contains('MIKE LAMB') |
        expenses.name.str.contains('LAMB FOR PA') |
        expenses.name.str.contains('MICHAEL LAMB'))
    ) | (
        expenses.address1.notna() & (expenses.address1.str.contains(' BOX 1835') | expenses.address1.str.contains('1015 GRANDVIEW'))
    )
]

In [9]:
committees = pd.read_csv(
    'input/committees.csv',
    dtype={'Committee Number': 'str'},
    usecols=['Committee Number', 'Committee Name'])
committees = committees.drop_duplicates('Committee Number', keep='first')

df_lamb_topcontribs = df_lamb.groupby('filer_id').amount_lamb.sum().to_frame()
df_lamb_topcontribs = pd.merge(
    df_lamb_topcontribs,
    committees,
    left_on='filer_id',
    right_on='Committee Number')

lamb_sum = df_lamb.amount_lamb.sum()
df_lamb_topcontribs['pct_lamb'] = df_lamb_topcontribs.amount_lamb / lamb_sum

df_lamb_topcontribs = df_lamb_topcontribs.rename(columns={'Committee Number': 'number', 'Committee Name': 'name'})
df_lamb_topcontribs[['number', 'name', 'amount_lamb', 'pct_lamb']].sort_values(by='amount_lamb', ascending=False).head(10)

,number,name,amount_lamb,pct_lamb
40,8400326,LOCAL 0005 IBEW PAC,56250.0,0.142927
32,8000488,LOCAL 0027 PLUMBERS UNION PAC,48550.0,0.123362
45,8600238,LOCAL 0449 STEAMFITTERS UNION PAC,40350.0,0.102526
43,8600169,LOCAL 0066 PAC CLUB,37000.0,0.094014
27,7900296,LOCAL 0690 PLUMBERS UNION POL ACTION FUND,27500.0,0.069875
35,8100217,LOCAL 0542 IUOE OPER ENG (NORRISTN),25000.0,0.063523
56,9500250,GREATER PA CARPENTERS PAC,18050.0,0.045864
39,8200660,INTL UNION PAINTERS ALLIED TRADE,14000.0,0.035573
20,20190173,WESTERN PENNSYLVANIA LABORERS 2019 PAC,10000.0,0.025409
52,9300158,INTL BROTHERHOOD BOILERMAKERS 154,7100.0,0.018041


### Liv Bennett (local committee)

### Merge common donors

In [10]:
dfs = [df_weinstein_topcontribs, df_innamorato_topcontribs, df_lamb_topcontribs]
dfs = [df.set_index('name') for df in dfs]
joined = reduce(lambda x, y: pd.merge(x,y, on='name', how='outer'), dfs)

In [11]:
joined[(joined.amount_wein.notna()) & (joined.amount_inna.notna()) & (joined.amount_lamb.notna())]

,amount_wein,pct_wein,amount_inna,pct_inna,amount_lamb,number,pct_lamb
name,,,,,,,
AFSCME COUNCIL 13 POL & LEG ACCT,350.0,0.000702,2500.0,0.007762,200.0,7900006,0.000508
LOCAL 0249 TEAMSTERS DRIVE,1000.0,0.002005,1500.0,0.004657,1050.0,7900662,0.002668
WESTERN PENNSYLVANIA LABORERS 2019 PAC,90000.0,0.180471,2500.0,0.007762,10000.0,20190173,0.025409


In [12]:
joined[(joined.amount_wein.isna()) & (joined.amount_inna.notna()) & (joined.amount_lamb.notna())]

,amount_wein,pct_wein,amount_inna,pct_inna,amount_lamb,number,pct_lamb
name,,,,,,,
1776 PAC (UFCW),NaN,NaN,750.0,0.002329,200.0,8200631,0.000508
DENTONS COHEN AND GRIGSBY PAC,NaN,NaN,750.0,0.002329,500.0,8500209,0.001270
GREATER PA CARPENTERS PAC,NaN,NaN,1000.0,0.003105,18050.0,9500250,0.045864
MALADY & WOOTEN PAC,NaN,NaN,250.0,0.000776,2750.0,9700250,0.006988
PFT POL ACTION FUND (PGH FED TEACHERS),NaN,NaN,3500.0,0.010867,2000.0,7900476,0.005082
SEIU HEALTHCARE PA COPE,NaN,NaN,11250.0,0.034929,50.0,20150138,0.000127


In [13]:
joined[(joined.amount_wein.notna()) & (joined.amount_inna.isna()) & (joined.amount_lamb.notna())]

,amount_wein,pct_wein,amount_inna,pct_inna,amount_lamb,number,pct_lamb
name,,,,,,,
LOCAL 0005 IBEW PAC,10000.0,0.020052,NaN,NaN,56250.0,8400326,0.142927
LOCAL 0009 BRICKLAYERS & ALLIED CRAFTWORKERS PA PAC,500.0,0.001003,NaN,NaN,7000.0,2000142,0.017786
LOCAL 0027 PLUMBERS UNION PAC,1000.0,0.002005,NaN,NaN,48550.0,8000488,0.123362
LOCAL 0433 UWUA (UTILITY WORKERS),500.0,0.001003,NaN,NaN,1000.0,9600256,0.002541
LOCAL 0449 STEAMFITTERS UNION PAC,15000.0,0.030078,NaN,NaN,40350.0,8600238,0.102526
PGH FIRE FIGHTERS LOCAL #1 FIRE PAC,2000.0,0.004010,NaN,NaN,5000.0,8600183,0.012705
SALVO PAC,1000.0,0.002005,NaN,NaN,3100.0,20170097,0.007877
TEAMSTERS JT COUNCIL 40 PAC,200.0,0.000401,NaN,NaN,500.0,9100226,0.001270
TEAMSTERS LOCAL 205 PAC,250.0,0.000501,NaN,NaN,200.0,20130205,0.000508
